In [29]:
import numpy as np
import pandas as pd
import arviz as az
import pymc3 as pm
import matplotlib.pyplot as plt
from causalgraphicalmodels import CausalGraphicalModel
import daft
from scipy import stats
from theano import shared
import re

**1. Entropy values**   
$-\sum_{i=1}^n p_i log(p_i)$

In [13]:
bird_data = {'bird_a': [.2, .8, .05], 'bird_b': [.2, .1, .15], 'bird_c': [.2, .05, .7], 'bird_d': [.2, .025, .05], 'bird_e': [.2, .025, .05]}
bird_df = pd.DataFrame(bird_data, index=['island_1', 'island_2', 'island_3'])
bird_df['entropy'] = -1 * (np.log(bird_df)*bird_df).sum(axis=1)
bird_df

,bird_a,bird_b,bird_c,bird_d,bird_e,entropy
island_1,0.20,0.20,0.20,0.200,0.200,1.609438
island_2,0.80,0.10,0.05,0.025,0.025,0.743004
island_3,0.05,0.15,0.70,0.050,0.050,0.983600


Island 1 has the most uncertainty in its distribution of birds since each bird type is equal. The other two islands has less uncertainty due to a large proportion of birds occuring for a particular type. 

**KL Divergence**   
$\sum_i p_i \left( \frac{p_i}{q_i} \right)$

In [57]:
def calc_kl_div(base_island):
    pattern_find = re.findall('\d$', base_island)
    base_num = pattern_find[0]
    islands = set(['island_1', 'island_2', 'island_3'])
    island_df = bird_df.loc[:, ['bird_a', 'bird_b', 'bird_c', 'bird_d', 'bird_e']].T
    div_df = pd.DataFrame()
    for island in islands - set([base_island]):
        inum = re.findall('\d$', island)[0]
        div_df[f'KLD({base_num}, {inum})'] = [np.sum(island_df[base_island] * np.log(island_df[base_island] / island_df[island]))]
        
    return div_df

In [58]:
klds = [calc_kl_div(f'island_{i}') for i in range(1, 4)]
kld_df = pd.concat(klds, axis=1)

In [62]:
kld_df.loc[:, ['KLD(2, 1)', 'KLD(3, 1)', 'KLD(1, 2)', 'KLD(3, 2)', 'KLD(1, 3)', 'KLD(2, 3)']]

,"KLD(2, 1)","KLD(3, 1)","KLD(1, 2)","KLD(3, 2)","KLD(1, 3)","KLD(2, 3)"
0,0.866434,0.625838,0.970406,1.838845,0.63876,2.010914


Island 1 predicts the other islands the best        

**2.** 

In [63]:
def inv_logit(x):
    return np.exp(x) / (1 + np.exp(x))


def sim_happiness(N_years=100, seed=1234):
    np.random.seed(seed)

    popn = pd.DataFrame(np.zeros((20 * 65, 3)), columns=["age", "happiness", "married"])
    popn.loc[:, "age"] = np.repeat(np.arange(65), 20)
    popn.loc[:, "happiness"] = np.repeat(np.linspace(-2, 2, 20), 65)
    popn.loc[:, "married"] = np.array(popn.loc[:, "married"].values, dtype="bool")

    for i in range(N_years):
        # age population
        popn.loc[:, "age"] += 1
        # replace old folk with new folk
        ind = popn.age == 65
        popn.loc[ind, "age"] = 0
        popn.loc[ind, "married"] = False
        popn.loc[ind, "happiness"] = np.linspace(-2, 2, 20)

        # do the work
        elligible = (popn.married == 0) & (popn.age >= 18)
        marry = (
            np.random.binomial(1, inv_logit(popn.loc[elligible, "happiness"] - 4)) == 1
        )
        popn.loc[elligible, "married"] = marry

    popn.sort_values("age", inplace=True, ignore_index=True)

    return popn

In [77]:
popn = sim_happiness()
adults = popn.loc[popn.age > 17, :]
adults = adults.assign(A=(adults.age-18)/(65-18))

In [79]:
mid = pd.Categorical(adults.loc[:, "married"].astype(int))

with pm.Model() as m_6_9:
    a = pm.Normal("a", 0, 1, shape=2)
    bA = pm.Normal("bA", 0, 2)

    mu = a[mid] + bA * adults.A.values
    sigma = pm.Exponential("sigma", 1)

    happiness = pm.Normal("happiness", mu, sigma, observed=adults.happiness.values)

    m_6_9_trace = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, bA, a]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 7 seconds.


In [80]:
with pm.Model() as m6_10:
    a = pm.Normal("a", 0, 1)
    bA = pm.Normal("bA", 0, 2)

    mu = a + bA * adults.A.values
    sigma = pm.Exponential("sigma", 1)

    happiness = pm.Normal("happiness", mu, sigma, observed=adults.happiness.values)

    trace_6_10 = pm.sample(1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, bA, a]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 4 seconds.


In [86]:
m_6_9.name = "model_6_9"
m6_10.name = "model_6_10"
az.compare({m_6_9: m_6_9_trace, m6_10: trace_6_10}, ic='waic', scale='deviance')

/home/jonathan/.conda/envs/rethinking/lib/python3.8/site-packages/arviz/data/io_pymc3.py:87: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(
/home/jonathan/.conda/envs/rethinking/lib/python3.8/site-packages/arviz/data/io_pymc3.py:87: FutureWarning: Using `from_pymc3` without the model will be deprecated in a future release. Not using the model will return less accurate and less useful results. Make sure you use the model argument or call from_pymc3 within a model context.
  warnings.warn(


,rank,waic,p_waic,d_waic,weight,se,dse,warning,waic_scale
<pymc3.model.Model object at 0x7f06003f6070>,0,2711.07,3.72388,0,1,36.369,0,False,deviance
<pymc3.model.Model object at 0x7f05fe727c70>,1,3037.48,2.34798,326.414,4.95384e-45,27.1539,33.0142,False,deviance


Model 6_9 conditions on a collider but by WAIC is the model that will yield better predictions. This is reasonable as by conditioning on the collider has opened up age information into the happiness result. However WAIC does not say anything about causality and cannot be used to choose a better model for intervention. 

**3.**